This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

env_file = 'api.txt'
load_dotenv(env_file)
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample 
# of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data'
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, 
# and take note of how nested it is,
# as well as the overall structure
params = {
    'start_date':'2017-01-01',
    'end_date':'2017-12-31'
}
res = requests.get(url, params)
json_data_2017 = res.json()

In [5]:
# json_data_2017

In [6]:
# json_data_2017['dataset_data'].values()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
# dataset_data is a key with a nested dictionary
# json_data_2017 has ten keys
# limit
# transform
# column_index
# column_names
# start_date
# end_date
# frequency
# data
# collapse
# order

json_data_2017['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [8]:
data = json_data_2017['dataset_data']['data']
# print(data)

In [9]:
col_names = json_data_2017['dataset_data']['column_names']
print(col_names)
print(data[0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [10]:
# find all indices for types == None, replace with 0
none_indices = [index for index, sublist in enumerate(data) if sublist[1] is None]
print(none_indices)

# 169, 179, 180

[169, 179, 180]


In [26]:
# get the mean value for open prices
days = len(data)
mean_value = 0
total_open = 0
for sublist in data:
    if sublist[1] is not None:
        total_open += sublist[1]
        mean_value = total_open / days
        
print(mean_value)

43.33820715109573


In [27]:
# set default value of 'None' to the mean
# print(mean_value)

for sublist in data:
    if sublist[1] is None:
        print(sublist[1])
        sublist[1] = mean_value
        

In [29]:
#3 Calculate what the highest and lowest opening prices 
# were for the stock in this period.

open_max = 0
open_min = data[1][1]
for sublist in data:
    if sublist[1] > open_max:
        open_max = sublist[1]
    if sublist[1] < open_min and sublist[1] > 0:
        open_min = sublist[1]
    
print(f'The highest opening price was ${open_max:,.2f}.')
print(f'The lowest opening price was ${open_min:,.2f}.')

The highest opening price was $53.11.
The lowest opening price was $34.00.


In [14]:
#4 What was the largest change in any one day (based on High and Low price)?
max_diff = 0
for i,j in enumerate(data):
    diff = j[2] - j[3]
    if diff > max_diff:
        max_diff = diff
        
print(f'The largest change in price for any one day was ${max_diff:,.2f}.')

The largest change in price for any one day was $2.81.


In [15]:
#5 What was the largest change between any two days (based on Closing Price)?
j=0
largest_close_diff = 0

while j < len(data)-1:
    day1 = data[j][4]
    day2 = data[j+1][4]
    close_diff = day2 - day1
    if close_diff > largest_close_diff:
        largest_close_diff = close_diff
    j+=1
  

print(f'The largest change in price for any two consecutive days was ${largest_close_diff:,.2f}.')

The largest change in price for any two consecutive days was $2.56.


In [16]:
#6 What was the average daily trading volume during this year?
days = len(data)
total_volume = 0
for sublist in data:
    total_volume += sublist[6]
    
average = total_volume/days
print(f'The average daily trading volume for the year 2017 was {average:,.2f} shares.')

The average daily trading volume for the year 2017 was 89,124.34 shares.


In [17]:
#7 (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
trade_vol = []
for sublist in data:
    trade_vol.append(sublist[6])
    
mid = (len(trade_vol) - 1)//2
trade_vol = sorted(trade_vol)

print(f'The median volume traded during the year was {trade_vol[mid]} shares.')    

The median volume traded during the year was 76286.0 shares.
